# Case Study: Sentiment Analysis

First stage of our competition. 

The work to be done is as follows:
1. Feature extraction and baseline
    * read the dataset and understand it
    * put it in a format so that you can use `CountVectorizer` or`Tf-IDF` to extract the desired features
    * perform on the desired dates and preprocessing
    * use one of the classifiers you know to predict the polarity of different sentences
1. My first neural network
    * reuse the features already extracted 
    * proposed a neural network built with Keras
1. Hyper-parameter fitting
    * for the base line: adjust min_df, max_df, ngram, max_features + model's hyper-parameter
    * for the neural network: adjust batch size, number of layers and number of neuron by layers, use earlystop
1. Word embedding
    * stage 1: build a network that uses Keras embedding which is not language sensitive.
    * stage 2: build a network that simultaneously uses Keras embedding and the features extracted in the first weeks.
1. Recurrent neural network<span style="color:red">
    * stage 1: build a network that uses recurrent neurons. The classification will be done on the last hidden state 
    * stage 2: build a network that uses recurrent neurons. The classification will be done on the mean of all hidden state
    * stage 3: build a bi-directional network that uses recurrent neurons. The classification will be done on the mean of all hidden state
    </span>



**WARNING:** the dataset is voluminous, I can only encourage you to work first on a small part of it and only at the end, when the code is well debugged and that it is necessary to build the "final model", to use the whole dataset.

## 1. Read the dataset

Could you find below a proposal. You can complete them.

In [0]:
student = True

if student:
    ''' FIX THIS '''
    path = "../dataset/"  
else:
    path = "../dataset/"

In [0]:
import pandas as pd
import numpy as np

def read(file, nb_items=5):
   with open(file, 'r') as f:
       df = pd.DataFrame({'id':[],
                          'review':[],
                          'lang':[],
                          'sentiment':[]})
       sid = []
       for line in f.readlines():
           tokens = line.strip().split('\t')
           if tokens[0]=="meta":
               if sid!=[]:
                   df.loc[len(df)] = [sid,
                                      review,
                                      lang,
                                      sentiment]
                   if nb_items!=0 and len(df)==nb_items:
                       return df
               sid = tokens[1]
               sentiment = tokens[2]
               review = []
               lang = []
           elif tokens[0]!="":
               if len(tokens)==2:
                   review += [tokens[0]]
                   lang += [tokens[1]]
       return df

read(path+"train_conll.txt", 2).head()

id                                             review  \
0   3  [@, AdilNisarButt, pakistan, ka, ghra, tauq, h...   
1  41  [Madarchod, mulle, ye, mathura, me, Nahi, dikh...   

                                                lang sentiment  
0  [O, Hin, Hin, Hin, Hin, Hin, Eng, Eng, Eng, En...  negative  
1  [Hin, Hin, Hin, Hin, Hin, Hin, Hin, Hin, Hin, ...  negative

In [0]:
from sklearn.model_selection import train_test_split

if student:
    ''' You could read the whole dataset - spend a lot of time '''
    df_train = read(path+"train_conll.txt", 4000)
    ''' You can adjust valid part '''
    df_train, df_valid = train_test_split(df_train, test_size=0.5)
else:
    df_train = read(path+"train_conll.txt", 0)
    df_train, df_valid = train_test_split(df_train, test_size=0.1)
    df_test = read(path+"trial_conll.txt", 0)

In [0]:
''' Define X '''

X_train = df_train['review']
X_valid = df_valid['review']
if not student:
    X_test = df_test['review']

X_train.shape, X_valid.shape

((2000,), (2000,))

In [0]:
''' Define y '''

y_train = df_train['sentiment']
y_valid = df_valid['sentiment']
if not student:
    y_test = df_test['sentiment']

y_train.shape, y_valid.shape

((2000,), (2000,))

In [0]:
''' If necessary, encode the label '''
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_valid_enc = encoder.transform(y_valid)
if not student:
    y_test_enc = encoder.transform(y_test)

nb_classes = len(np.unique(y_train_enc))
nb_classes

3

## 2. Text normalisation

It's your job

In [0]:
''' Put your code here '''

# you can put all the text in lower case
# you can remove all english stop word
# you can remove all/some punctuation sign
# you can remove all number
# you can stem/lemitize the english word
# ...

' Put your code here '

## 3. Feature extraction

Could you find below a proposal. You can complete them.

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
''' learn about the parameter of CountVectorizer '''
# analyzer
# ngram_range
# max_df and min_df
# max_features
# binary
# ...

X_train_enc = cv.fit_transform(X_train.apply(' '.join)).toarray()
X_valid_enc = cv.transform(X_valid.apply(' '.join)).toarray()
if not student:
    X_test_enc = cv.transform(X_test.apply(' '.join)).toarray()

## 4. Word embedding with Keras

In [0]:
from keras.preprocessing.text import Tokenizer

FILTERS='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
VOCABULARY_SIZE = 4_000 # Max size of the vocabulary in review

# create the tokenizer
tokenizer = Tokenizer(num_words=VOCABULARY_SIZE,
              filters=FILTERS,
              lower=True,
              split=' ',
              oov_token="_UNK_")

# fit the tokenizer on the documents
tokenizer.fit_on_texts(X_train)

# encode documents
X_train_enc2 = tokenizer.texts_to_sequences(X_train)
X_valid_enc2 = tokenizer.texts_to_sequences(X_valid)
if not student:
    X_test_enc2 = tokenizer.texts_to_sequences(X_test)
X_train_enc2[0]

Using TensorFlow backend.


[2,
 3526,
 2,
 3527,
 3528,
 3529,
 26,
 805,
 62,
 2027,
 14,
 2028,
 78,
 39,
 806,
 1473,
 504,
 1138,
 460,
 12,
 50,
 9,
 6,
 7,
 4,
 3,
 8,
 5,
 3530]

In [0]:
from keras.preprocessing.sequence import pad_sequences

SEQUENCE_SIZE = 24     # Nombre de mots dans une review
PADDING_MODE = 'pre'
PADDING_VALUE = 0
TRUNCATING_MODE='post'

X_train_enc2 = pad_sequences(X_train_enc2,
                            maxlen=SEQUENCE_SIZE,
                            padding=PADDING_MODE,
                            truncating=TRUNCATING_MODE,
                            value=PADDING_VALUE)
X_valid_enc2 = pad_sequences(X_valid_enc2,
                            maxlen=SEQUENCE_SIZE,
                            padding=PADDING_MODE,
                            truncating=TRUNCATING_MODE,
                            value=PADDING_VALUE)
if not student:
    X_test_enc2 = pad_sequences(X_test_enc2,
                            maxlen=SEQUENCE_SIZE,
                            padding=PADDING_MODE,
                            truncating=TRUNCATING_MODE,
                            value=PADDING_VALUE)
X_train_enc2[0]

array([   2, 3526,    2, 3527, 3528, 3529,   26,  805,   62, 2027,   14,
       2028,   78,   39,  806, 1473,  504, 1138,  460,   12,   50,    9,
          6,    7], dtype=int32)

## 5. Build neural network

Could you find below a proposal. You can complete them.

In [4]:
import keras
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Flatten, LSTM
from keras import utils

EMBEDDING_SIZE = 50

# This returns a tensor
inputs = Input(shape=(SEQUENCE_SIZE,))

# Word embedding
embedding = Embedding(VOCABULARY_SIZE,
                      EMBEDDING_SIZE,
                      input_length=SEQUENCE_SIZE)(inputs)

# a layer instance is callable on a tensor, and returns a tensor
output_1 = LSTM(16, return_sequences=False, activation='relu')(embedding)
output_2 = Dense(8, activation='relu')(output_1)
predictions = Dense(nb_classes, activation='softmax')(output_2)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)

model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

utils.print_summary(model)

NameError: ignored

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

# simple early stopping
es = EarlyStopping(monitor='val_loss', mode='min', patience=15, verbose=1)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

history = model.fit(X_train_enc2, y_train_enc, epochs=500, validation_data=[X_valid_enc2, y_valid_enc], callbacks=[es, mc])
model.save("model.h5")

Instructions for updating:
Use tf.cast instead.
Train on 2000 samples, validate on 2000 samples
Epoch 1/500
2000/2000 [==============================] - 1s 529us/step - loss: 1.0985 - accuracy: 0.3420 - val_loss: 1.0980 - val_accuracy: 0.3685

Epoch 00001: val_accuracy improved from -inf to 0.36850, saving model to best_model.h5
Epoch 2/500
2000/2000 [==============================] - 1s 395us/step - loss: 1.0979 - accuracy: 0.3635 - val_loss: 1.0972 - val_accuracy: 0.3680

Epoch 00002: val_accuracy did not improve from 0.36850
Epoch 3/500
2000/2000 [==============================] - 1s 398us/step - loss: 1.0975 - accuracy: 0.3570 - val_loss: 1.0968 - val_accuracy: 0.3680

Epoch 00003: val_accuracy did not improve from 0.36850
Epoch 4/500
2000/2000 [==============================] - 1s 411us/step - loss: 1.0973 - accuracy: 0.3570 - val_loss: 1.0965 - val_accuracy: 0.3680

Epoch 00004: val_accuracy did not improve from 0.36850
Epoch 5/500
2000/2000 [==============================] - 1s 


Epoch 00039: val_accuracy did not improve from 0.36850
Epoch 40/500
2000/2000 [==============================] - 1s 413us/step - loss: 1.0954 - accuracy: 0.3570 - val_loss: 1.0946 - val_accuracy: 0.3680

Epoch 00040: val_accuracy did not improve from 0.36850
Epoch 41/500
2000/2000 [==============================] - 1s 404us/step - loss: 1.0953 - accuracy: 0.3590 - val_loss: 1.0946 - val_accuracy: 0.3680

Epoch 00041: val_accuracy did not improve from 0.36850
Epoch 42/500
2000/2000 [==============================] - 1s 400us/step - loss: 1.0953 - accuracy: 0.3575 - val_loss: 1.0944 - val_accuracy: 0.3685

Epoch 00042: val_accuracy did not improve from 0.36850
Epoch 43/500
2000/2000 [==============================] - 1s 399us/step - loss: 1.0951 - accuracy: 0.3585 - val_loss: 1.0943 - val_accuracy: 0.3690

Epoch 00043: val_accuracy improved from 0.36850 to 0.36900, saving model to best_model.h5
Epoch 44/500
2000/2000 [==============================] - 1s 410us/step - loss: 1.0951 - accu

2000/2000 [==============================] - 1s 439us/step - loss: 1.0904 - accuracy: 0.3920 - val_loss: 1.0909 - val_accuracy: 0.3915

Epoch 00079: val_accuracy improved from 0.38300 to 0.39150, saving model to best_model.h5
Epoch 80/500
2000/2000 [==============================] - 1s 435us/step - loss: 1.0899 - accuracy: 0.4215 - val_loss: 1.0917 - val_accuracy: 0.3770

Epoch 00080: val_accuracy did not improve from 0.39150
Epoch 81/500
2000/2000 [==============================] - 1s 430us/step - loss: 1.0899 - accuracy: 0.4030 - val_loss: 1.0910 - val_accuracy: 0.3800

Epoch 00081: val_accuracy did not improve from 0.39150
Epoch 82/500
2000/2000 [==============================] - 1s 411us/step - loss: 1.0896 - accuracy: 0.4110 - val_loss: 1.0906 - val_accuracy: 0.3810

Epoch 00082: val_accuracy did not improve from 0.39150
Epoch 83/500
2000/2000 [==============================] - 1s 397us/step - loss: 1.0893 - accuracy: 0.4075 - val_loss: 1.0904 - val_accuracy: 0.3825

Epoch 00083: 

2000/2000 [==============================] - 1s 431us/step - loss: 0.9857 - accuracy: 0.4950 - val_loss: 1.0218 - val_accuracy: 0.4580

Epoch 00117: val_accuracy did not improve from 0.51050
Epoch 118/500
2000/2000 [==============================] - 1s 428us/step - loss: 0.9588 - accuracy: 0.5185 - val_loss: 0.9760 - val_accuracy: 0.4995

Epoch 00118: val_accuracy did not improve from 0.51050
Epoch 119/500
2000/2000 [==============================] - 1s 409us/step - loss: 0.9738 - accuracy: 0.5115 - val_loss: 0.9671 - val_accuracy: 0.5215

Epoch 00119: val_accuracy improved from 0.51050 to 0.52150, saving model to best_model.h5
Epoch 120/500
2000/2000 [==============================] - 1s 412us/step - loss: 0.9721 - accuracy: 0.5025 - val_loss: 0.9673 - val_accuracy: 0.5185

Epoch 00120: val_accuracy did not improve from 0.52150
Epoch 121/500
2000/2000 [==============================] - 1s 411us/step - loss: 0.9658 - accuracy: 0.5070 - val_loss: 1.0034 - val_accuracy: 0.4785

Epoch 001

Epoch 156/500
2000/2000 [==============================] - 1s 368us/step - loss: 0.7754 - accuracy: 0.6410 - val_loss: 0.9221 - val_accuracy: 0.5620

Epoch 00156: val_accuracy improved from 0.56000 to 0.56200, saving model to best_model.h5
Epoch 157/500
2000/2000 [==============================] - 1s 408us/step - loss: 0.7851 - accuracy: 0.6410 - val_loss: 0.9514 - val_accuracy: 0.5260

Epoch 00157: val_accuracy did not improve from 0.56200
Epoch 158/500
2000/2000 [==============================] - 1s 407us/step - loss: 0.7757 - accuracy: 0.6510 - val_loss: 0.9522 - val_accuracy: 0.5320

Epoch 00158: val_accuracy did not improve from 0.56200
Epoch 159/500
2000/2000 [==============================] - 1s 404us/step - loss: 0.7772 - accuracy: 0.6520 - val_loss: 0.9841 - val_accuracy: 0.4980

Epoch 00159: val_accuracy did not improve from 0.56200
Epoch 160/500
2000/2000 [==============================] - 1s 426us/step - loss: 0.7815 - accuracy: 0.6485 - val_loss: 0.9680 - val_accuracy: 0.5

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

# plot history
# list all data in history
print(history.history.keys())
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss train', 'loss val'], loc='upper left')
plt.show()

NameError: ignored

## 6. Evaluate

In [2]:
from sklearn.metrics import precision_recall_fscore_support
from keras.models import load_model
 
# load model
model = load_model('model.h5')
y_pred_enc = model.predict(X_valid_enc2)
y_pred_enc = np.argmax(y_pred_enc, axis=1)


precision, recall, fscore, support = precision_recall_fscore_support(y_valid_enc, y_pred_enc, average='weighted')
print("="*20, "validation set")
print("* P:", precision)
print("* R:", recall)
print("* F:", fscore)

if not student:
    y_pred_enc = model.predict(X_test_enc2)
    y_pred_enc = np.argmax(y_pred_enc, axis=1)
    precision, recall, fscore, support = precision_recall_fscore_support(y_test_enc, y_pred_enc, average='weighted')
    print("="*20, "test set")
    print("* P:", precision)
    print("* R:", recall)
    print("* F:", fscore)

Using TensorFlow backend.


OSError: ignored

## My score

**2000 items for training / 2000 items for validation**

* Logistic regression with BinaryVect without preprocessing
    * F: 0.558
* MLP [16, 8, 3] - 100 epochs with BinaryVect, without lemmatization
    * F: 0.561
* MLP [16, 8, 3] with early stopping without preprocessing
    * F: 0.569
* MLP [16, 8, 3] with lines=24, Keras embedding, early stopping without preprocessing
    * F: 0.541
* RNN [16] last hidden state - MLP[8, 3], with lines=24, Keras embedding, early stopping without preprocessing
    * F: 0.492